In [13]:
#P-GCS
from numpy import *
import matplotlib.pyplot as plt
from obj_funcs_lib import *

from scipy.optimize import minimize
import scipy
import numpy as np
np.set_printoptions(precision=2)
import datetime


In [14]:

def t(x):
    return np.square( np.tan(np.pi*x-np.pi/2) )
def T(x):
    return (np.pi/2-np.arctan( np.sqrt(x) ) )/np.pi
def s(x,a,b):
    return ((np.cos(2*np.pi*x-np.pi)+1)/2)*(T(b)-T(a))+T(a)
def w(x,a,b):
    return t(s(x,a,b))





In [15]:
OBJ_FUNCS

{'sphere': <function obj_funcs_lib.sphere(X)>,
 'rosenbrock': <function obj_funcs_lib.rosenbrock(X, a=1, b=100)>,
 'rastrigin': <function obj_funcs_lib.rastrigin(X)>,
 'ackley': <function obj_funcs_lib.ackley(X, a=20, b=0.2, c=6.283185307179586)>,
 'griewank': <function obj_funcs_lib.griewank(X)>,
 'schwefel': <function obj_funcs_lib.schwefel(X)>}

In [16]:
def optimize_with_pgcs(
    start_point, 
    objective,
    target=0.05, 
    w_a=0, 
    w_b=np.inf, 
    crunch_period=5000, 
    N=500000
):
    """
    Optimizes a given `objective` function starting from `start_point` with options for
    target value, step size parameters w_a, w_b, crunching period, and the number of iterations N.
    """
    sd = w_a
    current_pos = np.array(start_point)
    current_obj_value = objective(current_pos)
    crunch_step = 0
    w_cache = np.zeros(crunch_period)

    # Precompute w_cache
    for i in range(crunch_period):
        w_cache[i] = w(i / crunch_period, w_a, w_b)

    i = 0
    while i < N:
        if i == 0:
            powell_result = minimize(objective, current_pos, method='Powell')
            current_pos = powell_result.x
            current_obj_value = powell_result.fun
        temp_pos = current_pos + np.random.normal(0, sd, np.array(start_point).shape)
        temp_obj_value = objective(temp_pos)
        if temp_obj_value < current_obj_value:
            temp_powell = minimize(objective, temp_pos, method='Powell')
            temp_powell_pos = temp_powell.x
            temp_powell_value = temp_powell.fun
            if temp_powell_value < temp_obj_value:
                temp_obj_value = temp_powell_value
                temp_pos = temp_powell_pos
            current_pos = temp_pos
            current_obj_value = temp_obj_value
        else:
            crunch_step += 1
        if current_obj_value < target:
            break
        if crunch_step % crunch_period == 0:
            crunch_step = 0
        sd = w_cache[crunch_step]
        i += 1

    print(current_pos)
    print(objective(current_pos))
    return current_pos, objective(current_pos)


In [17]:
results = {}
for name, obj_func in OBJ_FUNCS.items():
    print(f"Optimizing {name}...")
    result = optimize_with_pgcs(
        start_point=[200]*3,
        objective=obj_func,
        
    )
    results[name] = result




Optimizing sphere...
[-1.99e-13 -2.56e-13 -2.56e-13]
1.704444426256727e-25
Optimizing rosenbrock...
[1. 1. 1.]
[9.97e-25]
Optimizing rastrigin...
[-8.67e-11 -3.67e-09 -3.67e-09]
0.0
Optimizing ackley...
[2.46e-12 4.95e-12 6.93e-13]
[1.29e-11]
Optimizing griewank...
[-1.04e-10 -3.46e-12  1.09e+01]
[0.03]
Optimizing schwefel...
[717.07 717.07 203.81]
-375.04307018055783
